This code will create a STAC catalog and add several items from a local folder

How do I put the date of image acquisition for an imagery product?  I don't think there is an automated way to do it. I need to manually input it. But how? Maybe I need to add metadata to imagery products within Cyverse and then read that metadata with pystack? How do I write metadata in Cyverse? 

In [1]:
%pip install rasterio shapely pystac

Note: you may need to restart the kernel to use updated packages.


In [2]:
from pathlib import Path
import pystac
import os

catalog = pystac.Catalog(
    id='Cyverse Remotely Sensed Imagery STAC Catalog',
    description='This catalog includes all of the imagery assets the exist in Cyverse Data Store',
    stac_extensions=['https://stac-extensions.github.io/projection/v1.0.0/schema.json']
)

This following code is a Python function called "get_bbox_and_footprint" that takes in a dataset (either from rasterio or rioxarray) and returns a tuple containing the bounding box of the dataset and the footprint of the dataset. The bounding box is represented as a list of 4 coordinates [left, bottom, right, top] and the footprint is represented as a shapely Polygon object.

First, the code imports the Polygon and mapping functions from the shapely.geometry library, and the rasterio library.

The function starts by extracting the bounds of the input dataset using the bounds attribute. The code then checks if the bounds are of type rasterio.coords.BoundingBox or not. If they are, it assigns the left, bottom, right and top coordinates of the bounding box to the bbox variable. If not, it creates a list of floating point numbers by iterating over the bounds and assigns it to the bbox variable.

Finally, the code creates a Polygon object using a list of the four corner coordinates of the bounding box. The footprint is obtained by mapping the Polygon object.

The function returns a tuple containing the bounding box and the footprint of the dataset.

In [3]:
from shapely.geometry import Polygon, mapping
import rasterio

#This creates a function called 'get_bbox_and_footprint' 
def get_bbox_and_footprint(dataset):

    # create the bounding box it will depend if it comes from rasterio or rioxarray
    bounds = dataset.bounds

    if isinstance(bounds, rasterio.coords.BoundingBox):
        bbox = [bounds.left, bounds.bottom, bounds.right, bounds.top]
    else:
        bbox = [float(f) for f in bounds()]

    # create the 4 corners of the footprint
    footprint = Polygon([
        [bbox[0], bbox[1]],#left bottom
        [bbox[0], bbox[3]],#left top
        [bbox[2], bbox[3]],#right top
        [bbox[2], bbox[1]] #right bottom
    ])

    return bbox, mapping(footprint)

In [4]:

#from rasterio.warp import transform_bounds, transform_geom
from pystac.extensions.projection import AssetProjectionExtension

# get a list of geospatial files (in this case located in Cyverse data store)
folder = Path('/data-store/iplant/home/jgillan/STAC_drone')
files = list(folder.rglob('*.tif'))#find files that end with .tif

#loop through each .tif item in the folder and do several things
for file in files:

    # open the individual file with rasterio
    ds = rasterio.open(file)
    
    #apply the function to get the bounding box (left, bottom, right, top) and make a footprint rectangle
    bbox, footprint = get_bbox_and_footprint(ds)
    
    #the ID (name) for each indivual file
    idx = file.stem
    
    #input the date and time of original imagery acquisition
    from datetime import datetime
    year = 2019
    month = 5
    day = 25
    hour = 12
    
    #Create datetime object
    user_datetime = datetime(year, month, day, hour)
    
    
    #Create a STAC item for each individual file 
    item = pystac.Item(id=idx,
                 geometry=footprint,
                 bbox=bbox,
                 datetime=user_datetime,
                 stac_extensions=['https://stac-extensions.github.io/projection/v1.0.0/schema.json'],
                 properties={})
                    
    
    #add each STAC item to the STAC catalog
    catalog.add_item(item)
    
    #link the assets (the actual geotif locations) to the STAC items
    item = catalog.get_item(idx)
    
    item.add_asset(
        key='geotiff',
        asset=pystac.Asset(
            href=file.as_posix(),
            media_type=pystac.MediaType.GEOTIFF
        )
    )
    
    # extend the asset with projection extension
    asset_ext = AssetProjectionExtension.ext(item.assets['geotiff'])
    asset_ext.epsg = ds.crs.to_epsg()
    asset_ext.shape = ds.shape
    asset_ext.bbox = bbox
    asset_ext.geometry = footprint
    asset_ext.transform = [float(getattr(ds.transform, letter)) for letter in 'abcdef']
    
    
#print the number of STAC items that were added to the STAC catalog    
print(len(list(catalog.get_items())))

#Describe the items in the STAC catalog
catalog.describe()

5
* <Catalog id=Cyverse Remotely Sensed Imagery STAC Catalog>
  * <Item id=1a_g2_ortho>
  * <Item id=1a_g2_sept_CHM>
  * <Item id=1a_u6_ortho>
  * <Item id=california>
  * <Item id=open_aerial>


In [5]:
stac_directory="/data-store/iplant/home/jgillan/STAC_drone"

catalog.normalize_hrefs(os.path.join(stac_directory, 'stac_test1'))

catalog.make_all_asset_hrefs_relative()
catalog.save(catalog_type=pystac.CatalogType.SELF_CONTAINED)